In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh
from collections import defaultdict
import networkx as nx
from pyvista import themes
pv.set_plot_theme(themes.DocumentTheme())

In [2]:
# pv.global_theme.colorbar_horizontal.width = 10
# pv.global_theme.colorbar_horizontal.position_x = 5

pv.global_theme.font.size = 14
pv.global_theme.font.label_size = 14
pv.global_theme.font.title_size = 18

In [3]:
def calculate_centroid(points):
    return np.mean(points, axis=0)


def vari(mesh):
    # Initialize a dictionary to store neighbors for each point
    neighbors = defaultdict(set)
    signal = mesh['non-directional_WSS_@_t=1.1']

    # Loop over each cell and populate the neighbors dictionary
    for i in range(mesh.n_cells):
        cell = mesh.get_cell(i)
        cell_points = cell.point_ids
        for i in range(len(cell_points)):
            for j in range(i + 1, len(cell_points)):
                neighbors[cell_points[i]].add(cell_points[j])
                neighbors[cell_points[j]].add(cell_points[i])


    # Visualize Total Variation by highlighting edges with largest differences
    # Initialize an array to store the total variation values
    total_variation = np.zeros(mesh.number_of_points)

    # Compute the total variation for each vertex
    for point_id in range(mesh.number_of_points):
        # Get the signal values at the point and its neighbors
        point_signal = signal[point_id]
        neighbor_signals = [signal[neighbor] for neighbor in neighbors[point_id]]
        # Calculate the total variation
        total_variation[point_id] = np.sum(np.abs(point_signal - np.array(neighbor_signals)))

    # Assign the total variation values to the mesh
    mesh.point_data['total_variation'] = total_variation

    max_var_indices = np.argsort(total_variation)[-50:]
    return mesh.extract_points(max_var_indices, include_cells=True), max_var_indices

def find_smallest_set_to_connect(mesh, sub_mesh_points):
    # Step 1: Create a graph representation of the mesh
    edges = mesh.extract_all_edges()
    graph = nx.Graph()
    
    for i in range(mesh.number_of_cells):
        cell = mesh.get_cell(i)
        cell_points = cell.point_ids
        n_points = len(cell_points)
        for j in range(n_points):
            for k in range(j + 1, n_points):
                graph.add_edge(cell_points[j], cell_points[k])

    
    # Step 2: Identify sub-mesh nodes
    sub_mesh_nodes = set(sub_mesh_points)
    
    # Step 3: Find connected components in the sub-mesh
    sub_graph = graph.subgraph(sub_mesh_nodes)
    components = list(nx.connected_components(sub_graph))
    
    if len(components) <= 1:
        return sub_mesh_points  # The sub-mesh is already connected

    max_variation_point = sub_mesh_points[np.argmax(mesh['total_variation'][sub_mesh_points])]

    selected_component = None
    for component in components:
        if max_variation_point in component:
            selected_component = component
            break
    
    if not selected_component:
        return []  # No valid component found, this should not happen
    
    # Step 6: Compute the smallest set of points to connect the selected component
    selected_component = list(selected_component)
    points_to_connect = set(selected_component)
    
    for i in range(len(selected_component)):
        for j in range(i + 1, len(selected_component)):
            node1 = selected_component[i]
            node2 = selected_component[j]
            
            try:
                path = nx.shortest_path(graph, source=node1, target=node2)
                points_to_connect.update(path)
            except nx.NetworkXNoPath:
                continue

    return list(points_to_connect)


In [4]:
mypath = '../../data/MLgSA/wss/'
mypath1 = '../../data/MLgSA/New_wss/'

onlyfiles = np.asarray([f for f in listdir(mypath) if isfile(join(mypath, f))])
onlyfiles1 = np.asarray([f for f in listdir(mypath1) if isfile(join(mypath1, f))])

onlyfiles.sort()
onlyfiles1.sort()

print(len(onlyfiles), len(onlyfiles1))

short_list = np.asarray([s[5:23] if s[19] == 'l' else s[5:24] for s in onlyfiles])
short_list1 = np.asarray([s[5:23] if s[19] == 'l' else s[5:24] for s in onlyfiles1])

109 50


In [5]:
j = 1
if j <110:
    reader = pv.get_reader(mypath + onlyfiles[j])
else:
    reader = pv.get_reader(mypath1 + onlyfiles1[j-109])
mesh  = reader.read()

In [6]:
cols = mesh.array_names
cols

['longitudinal_WSS_@_t=1.1',
 'non-directional_WSS_@_t=1.1',
 'longitudinal_WSS_@_t=1.55',
 'non-directional_WSS_@_t=1.55']

In [7]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars = cols[0], show_edges=False, opacity=0.3, cmap='seismic')
#max_var_nodes = mesh.points[m1]
#plotter.add_points(max_var_nodes, color='red', point_size=5)
plt.style.use("seaborn-v0_8-bright") 
plotter.show()

Widget(value='<iframe src="http://localhost:63455/index.html?ui=P_0x232a6f9ae90_0&reconnect=auto" class="pyvis…

In [8]:
mesh1, m1 = vari(mesh)

In [9]:
plotter = pv.Plotter()
plotter.add_mesh(mesh, scalars = 'total_variation', show_edges=False, opacity=0.3, cmap='plasma')
max_var_nodes = mesh.points[m1]
# plotter.add_points(max_var_nodes, color='red', point_size=5)
plotter.show()

Widget(value='<iframe src="http://localhost:63455/index.html?ui=P_0x232a84d96c0_1&reconnect=auto" class="pyvis…

In [10]:
points_to_connect = find_smallest_set_to_connect(mesh, mesh1['vtkOriginalPointIds'])
connected_sub_mesh_points = set(mesh1['vtkOriginalPointIds']).union(points_to_connect)
connected_sub_mesh = mesh.extract_points(list(connected_sub_mesh_points))

In [11]:
plotter = pv.Plotter()
plotter.add_mesh(connected_sub_mesh, scalars = 'total_variation', show_edges=True, opacity=0.5, cmap='seismic')
plotter.show()

Widget(value='<iframe src="http://localhost:63455/index.html?ui=P_0x232a84da830_2&reconnect=auto" class="pyvis…

In [12]:
mesh

UnstructuredGrid (0x232a6f93a00)
  N Cells:    33610
  N Points:   16947
  X Bounds:   -3.899e+01, -8.223e+00
  Y Bounds:   -3.777e+01, -5.576e+00
  Z Bounds:   -8.791e+01, -2.561e+01
  N Arrays:   7